In [7]:
import pyarrow.parquet as pq
import pyarrow.dataset as pa_ds

ModuleNotFoundError: No module named 'pyarrow._dataset'

In [2]:
table = pq.read_table('nano_dy.parquet')

In [3]:
len(table.schema.names)

977

In [4]:
common_fields = table.schema.names[:3]
common_fields

['run', 'luminosityBlock', 'event']

In [5]:
part_1 = table.schema.names[3:490]
len(part_1)

487

In [6]:
part_2 = table.schema.names[490:]
len(part_2)

487

In [7]:
table_1 = table.select(part_1)
len(table_1.schema.names)

487

In [8]:
table_2 = table.select(part_2)
len(table_2.schema.names)

487

In [9]:
table_1 = table_1.append_column('run', table.column('run'))
table_1 = table_1.append_column('luminosityBlock', table.column('luminosityBlock'))
table_1 = table_1.append_column('event', table.column('event'))
len(table_1.schema.names)

490

In [10]:
table_2 = table_2.append_column('run', table.column('run'))
table_2 = table_2.append_column('luminosityBlock', table.column('luminosityBlock'))
table_2 = table_2.append_column('event', table.column('event'))
len(table_2.schema.names)

490

In [11]:
len(table_2.schema.names)

490

In [12]:
len(table_1.schema.names)

490

In [13]:
table_1.column('Muon_dxy').flatten()

 [
   [
     [],
     [],
     ...
     [],
     []
   ]
 ]]

In [14]:
pq.write_table(table_1, 'table_1.parquet')

In [15]:
pq.write_table(table_2, 'table_2.parquet')

In [16]:
# joined_table = table_1.join(table_2, ['run', 'luminosityBlock', 'event'])

In [17]:
# df1

In [18]:
# df2

In [19]:
%%time
df1 = table_1.to_pandas(split_blocks=True, self_destruct=True)
df2 = table_2.to_pandas(split_blocks=True, self_destruct=True)
df1.merge(df2, how='inner', on=['run', 'luminosityBlock', 'event'])

CPU times: user 66.5 ms, sys: 3.63 ms, total: 70.2 ms
Wall time: 70.8 ms


,CaloMET_phi,CaloMET_pt,CaloMET_sumEt,nElectron,Electron_deltaEtaSC,Electron_dr03EcalRecHitSumEt,Electron_dr03HcalDepth1TowerSumEt,Electron_dr03TkSumPt,Electron_dxy,Electron_dxyErr,...,Flag_chargedHadronTrackResolutionFilter,Flag_muonBadTrackFilter,Flag_BadChargedCandidateFilter,Flag_BadPFMuonFilter,Flag_BadChargedCandidateSummer16Filter,Flag_BadPFMuonSummer16Filter,Flag_trkPOG_manystripclus53X,Flag_trkPOG_toomanystripclus53X,Flag_trkPOG_logErrorTooManyClusters,Flag_METFilters
0,2.527344,41.062500,889.50,0,[],[],[],[],[],[],...,1,1,1,1,1,1,1,1,1,1
1,1.991211,36.437500,979.00,2,"[0.02116394, 0.025268555]","[0.3623047, 1.3320312]","[1.3828125, 2.6484375]","[0.0, 0.0]","[-0.0017671585, -0.00080251694]","[0.0013122559, 0.0022277832]",...,1,1,1,1,1,1,1,1,1,1
2,2.478516,55.406250,1302.00,0,[],[],[],[],[],[],...,1,1,1,1,1,1,1,1,1,1
3,0.176514,25.234375,844.00,1,[0.0017910004],[0.0],[0.0],[0.0],[-0.0037059784],[0.004211426],...,1,1,1,1,1,1,1,1,1,1
4,-1.515625,49.656250,837.50,0,[],[],[],[],[],[],...,1,1,1,1,1,1,1,1,1,1
5,-0.931152,55.781250,1463.00,1,[-0.023345947],[0.0],[0.0],[0.0],[1.6201172],[0.75],...,1,1,1,1,1,1,1,1,1,1
6,-0.454590,19.718750,734.50,0,[],[],[],[],[],[],...,1,1,1,1,1,1,1,1,1,1
7,-1.869141,21.156250,487.25,0,[],[],[],[],[],[],...,1,1,1,1,1,1,1,1,1,1
8,-0.601562,18.718750,576.00,1,[0.009048462],[0.0],[0.0],[0.0],[-0.00548172],[0.0024414062],...,1,1,1,1,1,1,1,1,1,1
9,-1.222656,37.843750,1026.00,1,[0.005809784],[0.0],[0.0],[0.0],[0.029632568],[0.0036315918],...,1,1,1,1,1,1,1,1,1,1


In [20]:
# import duckdb
# con = duckdb.connect()

In [21]:
# results = con.execute("SELECT * FROM table_1 LEFT JOIN table_2 ON table_1.run == table_2.run")

In [22]:
!pip install datafusion

You should consider upgrading via the '/Users/jayjeetchakraborty/Courses/CSE215-Databases/project/venv/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:
from pycylon import CylonContext, Table
import pyarrow as pa
import pyarrow.parquet as pq
import pandas
import time

--------------------------------------------------------------------------
No OpenFabrics connection schemes reported that they were able to be
used on a specific port.  As such, the openib BTL (OpenFabrics
support) will be disabled for this port.

  Local host:           node0
  Local device:         mlx4_0
  Local port:           1
  CPCs attempted:       udcm
--------------------------------------------------------------------------


In [2]:
t1 = pq.read_table('table_1.parquet')
t2 = pq.read_table('table_2.parquet')

In [3]:
%%time
ctx = CylonContext()
ct1 = Table(t1,ctx)
ct2 = Table(t2,ctx)

joined = ct1.join(ct2, 'inner', 'hash', on=['run', 'event', 'luminosityBlock'])
print(len(joined.to_arrow().schema.names))

980
CPU times: user 20.7 ms, sys: 336 µs, total: 21 ms
Wall time: 20.3 ms


In [4]:
%%time
df1 = t1.to_pandas(split_blocks=True, self_destruct=True)
df2 = t2.to_pandas(split_blocks=True, self_destruct=True)

rdf = df1.merge(df2, how='inner', on=['run', 'luminosityBlock', 'event'])
rt = pa.Table.from_pandas(rdf)
print(len(rt.schema.names))

978
CPU times: user 236 ms, sys: 1.19 ms, total: 237 ms
Wall time: 234 ms
